## <span style="color:#ff5f27">📝 Imports </span>

In [ ]:
from sentence_transformers import SentenceTransformer
from FlagEmbedding import FlagReranker

from functions.llm_chain import get_llm_chain
from functions.prompt_engineering import get_context_and_source
import config

import warnings
warnings.filterwarnings('ignore')

## <span style="color:#ff5f27;"> 🔮 Connecting to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()
mr = project.get_model_registry()

In [ ]:
# Retrieve the 'documents' feature view
feature_view = fs.get_feature_view(
    name="documents", 
    version=1,
)   

# Initialize serving
feature_view.init_serving(1)

In [ ]:
# Get the Mistral model from Model Registry
mistral_model = mr.get_model(
    name="mistral_model",
    version=1,
)

# Download the Mistral model files to a local directory
saved_model_dir = mistral_model.download()

## <span style="color:#ff5f27">⛓️ LLM Chain </span>

In [ ]:
llm_chain = get_llm_chain(saved_model_dir)

In [ ]:
session_id = {
    "configurable": {"session_id": "default"}
}

## <span style="color:#ff5f27">🗄️ Sentence Transformer Loading </span>

In [ ]:
# Load the Sentence Transformer
sentence_transformer = SentenceTransformer(
    config.MODEL_SENTENCE_TRANSFORMER,
).to(config.DEVICE)

## <span style="color:#ff5f27;"> 🧬 Reranking </span>

In [ ]:
def get_reranker():
    reranker = FlagReranker(
        'BAAI/bge-reranker-large', 
        use_fp16=True,
    ) 
    return reranker

In [ ]:
# Retrieve a reranker
reranker = get_reranker()

## <span style="color:#ff5f27;">🗄️ Context Retrieval </span>

In [ ]:
# User Question Example
user_input = 'What are the best risk reporting practices?' 

In [ ]:
# Retrieve reranked context and source
context, source = get_context_and_source(
    user_input, 
    sentence_transformer,
    feature_view, 
    reranker,
)

## <span style="color:#ff5f27;">🚀 Model Inference </span>

In [ ]:
# Generate model response
model_output = llm_chain.invoke({
        "context": context, 
        "question": user_input,
    },
    session_id,
)

print(model_output.split('### RESPONSE:\n')[-1] + source)

In [ ]:
user_input = 'What is Adaptability?'

context, source = get_context_and_source(
    user_input, 
    sentence_transformer,
    feature_view, 
    reranker,
)

model_output = llm_chain.invoke({
        "context": context, 
        "question": user_input,
    },
    session_id,
)

print(model_output.split('### RESPONSE:\n')[-1] + source)

In [ ]:
user_input = 'What is a risk management?'

context, source = get_context_and_source(
    user_input, 
    sentence_transformer,
    feature_view, 
    reranker,
)

model_output = llm_chain.invoke({
        "context": context, 
        "question": user_input,
    },
    session_id,
)

print(model_output.split('### RESPONSE:\n')[-1] + source)

In [ ]:
user_input = 'What is the purpose of maintaining an up-to-date data-flow diagram?'

context, source = get_context_and_source(
    user_input, 
    sentence_transformer,
    feature_view, 
    reranker,
)

model_output = llm_chain.invoke({
        "context": context, 
        "question": user_input,
    },
    session_id,
)

print(model_output.split('### RESPONSE:\n')[-1] + source)

In [ ]:
user_input = 'Why are security and privacy controls important?'

context, source = get_context_and_source(
    user_input, 
    sentence_transformer,
    feature_view, 
    reranker,
)

model_output = llm_chain.invoke({
        "context": context, 
        "question": user_input,
    },
    session_id,
)

print(model_output.split('### RESPONSE:\n')[-1] + source)

---